In [1]:
%load_ext autoreload
%autoreload 2

# Metal ion transmembrane transporter activity
GO:0046873, GO idx 547 

In [2]:
from pathlib import Path
from omegaconf import OmegaConf

import torch
from plaid.diffusion import FunctionOrganismDiffusion
from plaid.denoisers import FunctionOrganismUDiT, DenoiserKwargs
from plaid.constants import COMPRESSION_INPUT_DIMENSIONS, COMPRESSION_SHORTEN_FACTORS

device = torch.device("cuda")

/homefs/home/lux70/code/plaid/src/plaid/denoisers/modules/_attention.py:12: FutureWarning: xformers.components is deprecated and is not maintained anymore. It might be removed in a future version of xFormers 
  from xformers.components.attention import ScaledDotProduct


In [3]:
model_id = "5j007z42"

ckpt_dir = Path("/data/lux70/plaid/checkpoints/plaid-compositional") 
model_path = ckpt_dir / model_id / "last.ckpt"
config_path = ckpt_dir / model_id / "config.yaml"

cfg = OmegaConf.load(config_path)

In [4]:
compression_model_id = cfg['compression_model_id']
shorten_factor = COMPRESSION_SHORTEN_FACTORS[compression_model_id]
input_dim = COMPRESSION_INPUT_DIMENSIONS[compression_model_id]

In [5]:
denoiser_kwargs = cfg.denoiser
denoiser_kwargs.pop("_target_")
denoiser = FunctionOrganismUDiT(**denoiser_kwargs, input_dim=input_dim)

In [6]:
# lask.ckpt automatically links to the EMA

ckpt = torch.load(model_path)
ckpt.keys()

/tmp/ipykernel_126960/2446128762.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path)


dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])

In [7]:
mod_state_dict = {}
for k, v in ckpt['state_dict'].items():
    if k[:16] == "model._orig_mod.":
        mod_state_dict[k[16:]] = v

In [8]:
denoiser.load_state_dict(mod_state_dict)

<All keys matched successfully>

In [9]:
diffusion_kwargs = cfg.diffusion
diffusion_kwargs.pop("_target_")

# diffusion_kwargs['beta_scheduler_name'] = "sigmoid"
# diffusion_kwargs['sampling_timesteps'] = 500

diffusion = FunctionOrganismDiffusion(model=denoiser,**diffusion_kwargs)

In [10]:
from cheap.pretrained import load_model_from_id
cheap_model = load_model_from_id(compression_model_id)
_ = cheap_model.to(device)

from cheap.proteins import LatentToSequence,LatentToStructure
latent_to_sequence = LatentToSequence()
latent_to_sequence.to(device)

latent_to_structure = LatentToStructure()
latent_to_structure.to(device)

/homefs/home/lux70/code/cheap-proteins/src/cheap/pretrained.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(checkpoint_fpath)


Using tanh layer at bottleneck...
Finished loading HPCT model with shorten factor 2 and 32 channel dimensions.


/homefs/home/lux70/code/cheap-proteins/src/cheap/decoder.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path)
Creating ESMFold...
ESMFold model 

# Sample

In [11]:
# organism_idx = org_df[org_df.organism_id == "HUMAN"].organism_index.iloc[0]
# function_idx = go_df[go_df.GO_term == "carbohydrate metabolic process"].GO_idx.iloc[0]
# print(organism_idx, function_idx)

In [12]:
device = torch.device("cuda")
diffusion = diffusion.to(device)

In [26]:
from plaid.datasets import NUM_ORGANISM_CLASSES, NUM_FUNCTION_CLASSES
import numpy as np

N, L = 8, 180
assert L % 4 == 0

shape = (N, L, input_dim)

organism_idx = NUM_ORGANISM_CLASSES
function_idx = 547 

cond_scale = 10

by_scale = {}

for scale in np.arange(1, 10, 1):
    diffusion.sampling_timesteps=1000
    
    sampled_latent = diffusion.ddim_sample_loop(
        shape=shape,
        organism_idx=organism_idx,
        function_idx=function_idx,
        return_all_timesteps=True,
        cond_scale=float(scale),
    )
    by_scale[scale]=sampled_latent

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

In [13]:
!ls /data/lux70/plaid/artifacts/

by_length_GO0006352.pkl  interpolation	noise_schedule_figures
by_scale_GO0046873.pkl	 mask_tokens	samples
eval			 natural	zero_out_massive_activations


In [28]:
# # import pickle as pkl

# with open("/data/lux70/plaid/artifacts/by_scale_GO0046873.pkl", "wb") as f:
#     pkl.dump(by_scale, f)

In [15]:
import pickle as pkl
with open("/data/lux70/plaid/artifacts/by_scale_GO0046873.pkl", "rb") as f:
    by_scale = pkl.load(f)

In [124]:
print(by_scale.keys())
cond_scale=8
sampled_latent = by_scale[cond_scale]

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9])


In [125]:
print(sampled_latent.shape)
final_sample = sampled_latent[:, -1, :, :]
print(final_sample.shape)

print(final_sample.max(), final_sample.min())

torch.Size([8, 1001, 180, 32])
torch.Size([8, 180, 32])
tensor(1.0323, device='cuda:0') tensor(-1.0461, device='cuda:0')


In [126]:
sampled_uncompressed = cheap_model.decode(final_sample, downsampled_mask=None)
print(sampled_uncompressed.min(), sampled_uncompressed.max())

tensor(-2.2607, device='cuda:0') tensor(2.0964, device='cuda:0')


In [127]:
# import pickle as pkl
# with open("test_sample.pkl", "wb") as f:
#     pkl.dump(sampled_uncompressed,f)

In [128]:
from cheap.utils import LatentScaler
latent_scaler = LatentScaler()
sampled_unscaled = latent_scaler.unscale(sampled_uncompressed) 

In [129]:
print(sampled_unscaled.shape)
print(sampled_unscaled.max(), sampled_unscaled.min())

torch.Size([8, 360, 1024])
tensor(4350.7627, device='cuda:0') tensor(-1379.1875, device='cuda:0')


In [130]:
sequences = latent_to_sequence.to_sequence(sampled_unscaled)[-1]

In [131]:
sequences[:3]

['KATDLAGGSLFGYVLLTVVLASSLMCIFLQSMANRLGIATRQDLARACRDAYPKPVAFPLKIGVEIAVAAVDLAEVMGTAIALNLLFGLPLTWGVLLTILDVWIVLFLQKKGMRKIEALVVAFIHTVMGVFGFELFLAKADLGAIFKGFLPDPEIVTNKQLLHIAAGILGATVFPHDLYLHSSVVQTRNYGRSERDRKEANRFQTIDSTIALFMALLINAAILIVASAAFHTTGRTDVIELEHAHRLLSPVLGVGLASTIFGVALLASGLDSTISGTLAGQSIFEGFLELRLKPWVRRLLTRLIAIIPSLAVTAIRGDRGHTELLVLSQVILSLQLPFAVIPLITLSSSSTKMGELNSNL',
 'RATGLDGGSTYGYTLLWVILLGSLLAMLLLSLSALLGTVTGRDLSEVSRDDYSRWSGMILWILSELAIIATDLPETIGTAIALNLLLGIPLWVGTLITGLGTLLILLLDNRGARKLGLLICTMVATVGAAFAVELSEVKPQSGDVERGLVCTTDVVSGKAMLWCSTGVLGSTVMPHGAYLHSSVTQTRRVSTSMCGVKDAVRHFTIESGANLFMSFFINASTLIASGGGYWKTGKTDVGGLQDAKHLLDPILGSSVASVLYGIGLLASGQSSTITGTLAGGTVFGGFLGFTLGSRVRRLITRGCSIIPGLIVVSFVGHNGSGELLVLSQVVLSLGLPFSTVPLVKFTSSKTKMGTMASGV',
 'KATDLAGGSQFGYVLLSVVLLSSLMIIFLQNMAGSLGIATQEDLAQACRLSYNKKVAFGLWLLVELAVAAHDLAEVMGGAIGMHLLSGLPLKPGVIITVLDVLLVLLLHDKGMRKIEAIVVGFIFLILGVFTFELILAKPNVGQIARGFVPKAEIVTNKNLLFIALGILGATVMPQDLYLGSSVVQTRDFDKTPYGKKEAIKFATIDSTIALLLAFLINNAILVLAAACFYRSGKTDVKEIENAYKLLNPVLGVSGSSTLFGCALLCS

In [132]:
pdb_strs = latent_to_structure.to_structure(sampled_unscaled, sequences=sequences, batch_size=8, num_recycles=4)

(Generating structure): 100%|██████████████| 1/1 [01:20<00:00, 80.16s/it]


In [133]:
import py3Dmol

for i in range(len(pdb_strs)):
# for i in range(10,20): 
# for i in range(0, 10): 
    view = py3Dmol.view(width=600, height=600)
    view.addModelsAsFrames(pdb_strs[i])
    
    # Apply the plDDT color scheme
    # view.setStyle({'cartoon': {'color': {'prop': 'b', 'gradient': 'roygb', 'min': 0, 'max': 100}}})
    view.setStyle({'cartoon': {'color': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}}})
    
    # # Add surface representation with plDDT-based color
    view.addSurface(py3Dmol.VDW, {'opacity': 0.7, 'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}})
    # view.addSurface(py3Dmol.VDW, {'opacity': 0.7, 'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 0, 'max': 100}})

    view.zoomTo()
    view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [134]:
from pathlib import Path

samples_dir = Path("/data/lux70/plaid/artifacts/GO_0036873/by_scale")
if not samples_dir.exists():
    samples_dir.mkdir(parents=True)

for i in range(len(pdb_strs)):
    with open(samples_dir / f"condscale{cond_scale}_sample{i}", "w") as f:
        f.write(pdb_strs[i])

In [135]:
print(cond_scale)

8


In [136]:
!ls /data/lux70/plaid/artifacts/GO_0036873/by_scale

condscale1_sample0  condscale3_sample2	condscale5_sample4  condscale7_sample6
condscale1_sample1  condscale3_sample3	condscale5_sample5  condscale7_sample7
condscale1_sample2  condscale3_sample4	condscale5_sample6  condscale8_sample0
condscale1_sample3  condscale3_sample5	condscale5_sample7  condscale8_sample1
condscale1_sample4  condscale3_sample6	condscale6_sample0  condscale8_sample2
condscale1_sample5  condscale3_sample7	condscale6_sample1  condscale8_sample3
condscale1_sample6  condscale4_sample0	condscale6_sample2  condscale8_sample4
condscale1_sample7  condscale4_sample1	condscale6_sample3  condscale8_sample5
condscale2_sample0  condscale4_sample2	condscale6_sample4  condscale8_sample6
condscale2_sample1  condscale4_sample3	condscale6_sample5  condscale8_sample7
condscale2_sample2  condscale4_sample4	condscale6_sample6  condscale9_sample0
condscale2_sample3  condscale4_sample5	condscale6_sample7  condscale9_sample1
condscale2_sample4  condscale4_sample6	condscale7_sample0  conds